In [1]:
! pip install wandb

## Import Libraries & Load Dataset

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import wandb
from wandb.keras import WandbCallback

In [ ]:
configs = {
              "learning_rate": 0.001,
              "epochs": 20,
              "batch_size": 32,
              "log_step": 200,
              "val_log_step": 50
           }

run = wandb.init(project='Face_Recognition', config=configs)
config = wandb.config

## Face Recognition Dataset

In [4]:
image_generator = ImageDataGenerator(
    rescale = 1.0 / 255.0,
    horizontal_flip = True,
    validation_split = 0.2
)

In [5]:
path = "/content/drive/MyDrive/Dataset/7-7 dataset"
Train_images = image_generator.flow_from_directory(
    path,
    class_mode = "categorical",
    batch_size = 32,
    target_size = (224,224),
    subset = "training"
)
Val_images = image_generator.flow_from_directory(
    path,
    class_mode = "categorical",
    batch_size = 32,
    target_size = (224,224),
    subset = "validation"
)

Found 1090 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


# Model
## CNN

In [9]:
class Mymodel(Model):
  def __init__(self):
      super().__init__()
      self.conv2D_1 = Conv2D(64,(3,3),activation="relu",input_shape=(224,224,3))
      self.conv2D_2 = Conv2D(128,(3,3),activation="relu")
      self.maxpool2D = MaxPool2D()
      self.dense1 = Dense(256,activation="relu")
      self.dense2 = Dense(14,activation="softmax")
      self.flatten = Flatten()
      self.dropout = Dropout(0.5)
    
  def call(self,x):
    x = self.conv2D_1(x)
    x = self.maxpool2D(x)
    x = self.conv2D_2(x)
    x = self.maxpool2D(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dropout(x)
    output = self.dense2(x)

    return output
model = Mymodel()

In [10]:
optimizer = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.CategoricalCrossentropy()
accuracy = tf.keras.metrics.CategoricalAccuracy()
val_accuracy = tf.keras.metrics.CategoricalAccuracy()
loss = tf.keras.metrics.MeanAbsoluteError()
val_loss = tf.keras.metrics.MeanAbsoluteError()

In [11]:
# Train
for epoch in range(20):
  accuracy.reset_states()
  val_accuracy.reset_states()
  loss.reset_states()
  val_loss.reset_states()
  print("Epoch: ",epoch)

  # Training
  for index,(images, labels) in enumerate(tqdm(Train_images)):
    with tf.GradientTape() as gtape:
      logits = model(images)
      loss_value = loss_func(labels,logits)
      accuracy(labels,logits)
      loss(labels,logits)
    # Gradients calculation
    gradients = gtape.gradient(loss_value,model.trainable_variables)
    # Update weights
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    if len(Train_images) <= index:
      break
 
  # Validation
  for index,(images, labels) in enumerate(tqdm(Val_images)):
    logits = model(images)
    loss_value = loss_func(labels,logits)
    val_accuracy(labels,logits)
    val_loss(labels,logits)
    if len(Val_images) <= index:
      break

  print("Accuracy: ",accuracy.result())
  print("Val_Accuracy: ",val_accuracy.result())
  print("Loss: ",loss.result())
  print("Val_Loss: ",val_loss.result())

  wandb.log({'epochs': epoch,
            'loss': np.mean(loss.result()),
            'acc': float(accuracy.result()), 
            'val_loss': np.mean(val_loss.result()),
            'val_acc':float(val_accuracy.result())})
  

Epoch:  0


100%|██████████| 9/9 [01:01<00:00,  6.85s/it]


Accuracy:  tf.Tensor(0.23083779, shape=(), dtype=float32)
Val_Accuracy:  tf.Tensor(0.40666667, shape=(), dtype=float32)
Loss:  tf.Tensor(0.1238171, shape=(), dtype=float32)
Val_Loss:  tf.Tensor(0.10209479, shape=(), dtype=float32)
Epoch:  1


100%|██████████| 9/9 [00:06<00:00,  1.34it/s]


Accuracy:  tf.Tensor(0.7076649, shape=(), dtype=float32)
Val_Accuracy:  tf.Tensor(0.59, shape=(), dtype=float32)
Loss:  tf.Tensor(0.06676322, shape=(), dtype=float32)
Val_Loss:  tf.Tensor(0.07080748, shape=(), dtype=float32)
Epoch:  2


...

Accuracy:  tf.Tensor(1.0, shape=(), dtype=float32)
Val_Accuracy:  tf.Tensor(0.85714287, shape=(), dtype=float32)
Loss:  tf.Tensor(4.9485676e-05, shape=(), dtype=float32)
Val_Loss:  tf.Tensor(0.021898568, shape=(), dtype=float32)
Epoch:  18


100%|██████████| 9/9 [00:06<00:00,  1.32it/s]


Accuracy:  tf.Tensor(1.0, shape=(), dtype=float32)
Val_Accuracy:  tf.Tensor(0.8566667, shape=(), dtype=float32)
Loss:  tf.Tensor(3.2558964e-05, shape=(), dtype=float32)
Val_Loss:  tf.Tensor(0.02110424, shape=(), dtype=float32)
Epoch:  19


100%|██████████| 9/9 [00:06<00:00,  1.29it/s]

Accuracy:  tf.Tensor(1.0, shape=(), dtype=float32)
Val_Accuracy:  tf.Tensor(0.87333333, shape=(), dtype=float32)
Loss:  tf.Tensor(3.2069842e-05, shape=(), dtype=float32)
Val_Loss:  tf.Tensor(0.019878231, shape=(), dtype=float32)


In [13]:
model.save_weights(filepath='face_recognition')